In [10]:
import pandas as pd
import os

# Define paths to directories and files
old_data_dir = 'old_data'
new_data_file = 'data_v2/clinic_genetic.csv'



In [11]:
# Read all CSVs in the old data directory and concatenate
old_dataframes = []
for file in os.listdir(old_data_dir):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(old_data_dir, file))
        old_dataframes.append(df)

# Concatenate all old dataframes, allowing for differing columns
old_data = pd.concat(old_dataframes, ignore_index=True)

# Group by 'title' and 'abstract', keeping the first occurrence but detecting differing rows
grouped_old_data = old_data.groupby(['title',], as_index=False).first()
for _, group in old_data.groupby(['title',]):
    if len(group) > 1:
        differing_rows = group.loc[:, group.nunique() > 1]
        if not differing_rows.empty:
            print("Differing rows for title/abstract group:\n", differing_rows)

Differing rows for title/abstract group:
     relevance
191       1.0
234         1
Differing rows for title/abstract group:
     relevance
175       1.0
235         1
Differing rows for title/abstract group:
     relevance
219       2.0
257         2
Differing rows for title/abstract group:
     relevance
187       1.0
233         1
Differing rows for title/abstract group:
     relevance
9         2.0
258         2
Differing rows for title/abstract group:
     relevance
177       1.0
236         1
Differing rows for title/abstract group:
     relevance
10        2.0
259         2
Differing rows for title/abstract group:
     relevance
210       2.0
260         2
Differing rows for title/abstract group:
     relevance                                           ai_topic
218       1.0                         ChatGPT; systematic review
252         1  ChatGPT; ML Task: Applications in bioinformati...
Differing rows for title/abstract group:
     relevance
201       2.0
264         2
Differi

In [12]:
cols_to_use = ['title', 'abstract', 'source', 'relevance', 'ai_topic',
       'medicine_topic', 'notes', 'used?', 'what section used']
grouped_old_data = grouped_old_data[cols_to_use]

print(grouped_old_data.shape)
grouped_old_data

(229, 9)


,title,abstract,source,relevance,ai_topic,medicine_topic,notes,used?,what section used
0,"""Having cancer is very expensive"": A qualitati...",To examine patient barriers and facilitators t...,PubMed,0.0,None,None,None,None,None
1,A Combined Manual Annotation and Deep-Learning...,We report a combined manual annotation and dee...,PubMed,2,BERT-based and DistilBERT-based NER models; ML...,Entity extraction in hereditary disease-relate...,BERT-based NER achieved high F1-scores for gen...,1,"pre: KNLR, disc"
2,A Combined Manual Annotation and Deep-Learning...,We report a combined manual annotation and dee...,PubMed,2.0,BERT-based and DistilBERT-based NER models; ML...,Entity extraction in hereditary disease-relate...,BERT-based NER achieved high F1-scores for gen...,None,None
3,A Comparative Sentiment Analysis of Greek Clin...,In addressing the critical role of emotional c...,PubMed,1.0,"BERT, RoBERTa, GPT-2, XLNet; ML Task: Sentimen...",Sentiment analysis of patient-clinician conver...,Importance of different language incorporation...,None,None
4,A Metabolic Biomarker Panel for Congenital Hea...,BackgroundCongenital heart disease (CHD) repre...,medrxiv,0.0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
224,Utilization of a Third-party Partnership in Te...,To meet the increasing demands of genetic risk...,PubMed,0.0,None,None,None,None,None
225,VarChat: the generative AI assistant for the i...,"In the modern era of genomic research, the sci...",PubMed,2.0,Generative AI; ML Task: Literature search and ...,Interpretation of genomic variants using summa...,VarChat summarizes genomic variant data from s...,1,ana: GVI
226,Virtual Labs and Designer Bugs - Generative AI...,AI technologies can pose a major national secu...,PubMed,1.0,No specific transformer-based models like GPT ...,Genetic research with a focus on synthetic bio...,AI's role in genetic research and biological s...,None,None
227,Weakly Supervised Classification for Nasophary...,Pathological examination of nasopharyngeal car...,PubMed,1.0,Vision Transformer (T2T-ViT); ML Task: Image c...,Cancer diagnosis using whole slide images (WS...,Proposes a weakly supervised transformer frame...,None,None


In [18]:
# Read the new data
new_data = pd.read_csv(new_data_file)

# Add missing columns from old data to new data
for col in grouped_old_data.columns:
    if col not in new_data.columns:
        new_data[col] = None

# Update values in new data with old data where title and abstract match
merged_data = new_data.merge(
    grouped_old_data, on=['title'], how='left', suffixes=('', '_old')
)
for col in grouped_old_data.columns:
    if col not in ['title']:
        merged_data[col] = merged_data[col].combine_first(merged_data[col + '_old'])

# Add a new column to indicate existence in old data
merged_data['existed_in_old_data'] = merged_data['title'].isin(grouped_old_data['title']) 

# Create an extra dataframe with rows not in the new data based on 'title' and 'abstract'
extra_data = grouped_old_data[
    ~grouped_old_data.set_index(['title']).index.isin(
        new_data.set_index(['title']).index
    )
]

In [19]:
merged_data

,title,abstract,source,relevance,ai_topic,medicine_topic,notes,used?,what section used,abstract_old,source_old,relevance_old,ai_topic_old,medicine_topic_old,notes_old,used?_old,what section used_old,existed_in_old_data
0,Natural Language Processing and Schizophrenia:...,(1) Background: Approximately 1% of the global...,PubMed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,Harnessing generative AI to annotate the sever...,0.1There are thousands of human phenotypes whi...,medrxiv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Leveraging hierarchical structures for genetic...,1.Initially introduced in 1909 by William Bate...,"PubMed,medrxiv",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,16S rRNA gene sequencing for bacterial identif...,16S rRNA gene sequence is the most common hous...,biorxiv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,A Systematic Review of Testing and Evaluation ...,1ImportanceLarge Language Models (LLMs) can as...,medrxiv,1.0,Various Large Language Models (LLMs) like GPT ...,"Healthcare applications, especially in clinica...",Evaluates LLMs in healthcare for diverse tasks...,NaN,NaN,1ImportanceLarge Language Models (LLMs) can as...,medrxiv,1.0,Various Large Language Models (LLMs) like GPT ...,"Healthcare applications, especially in clinica...",Evaluates LLMs in healthcare for diverse tasks...,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,Optogenetics and Targeted Gene Therapy for Ret...,With a common aim of restoring physiological f...,PubMed,0.0,None,None,None,None,None,With a common aim of restoring physiological f...,PubMed,0.0,None,None,None,None,None,True
754,VStrains: De Novo Reconstruction of Viral Stra...,"With the high mutation rate in viruses, a mixt...",biorxiv,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
755,Enhancing human phenotype ontology term extrac...,With the increasing utilization of exome and g...,PubMed,NaN,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
756,Towards a new standard in genomic data privacy...,With the rapid developments in sequencing tech...,biorxiv,0.0,None,None,None,None,None,With the rapid developments in sequencing tech...,biorxiv,0.0,None,None,None,None,None,True


In [22]:
sum(merged_data.existed_in_old_data)

124

In [26]:
extra_data[extra_data.relevance.astype(float)>0]

,title,abstract,source,relevance,ai_topic,medicine_topic,notes,used?,what section used
1,A Combined Manual Annotation and Deep-Learning...,We report a combined manual annotation and dee...,PubMed,2,BERT-based and DistilBERT-based NER models; ML...,Entity extraction in hereditary disease-relate...,BERT-based NER achieved high F1-scores for gen...,1,"pre: KNLR, disc"
2,A Combined Manual Annotation and Deep-Learning...,We report a combined manual annotation and dee...,PubMed,2.0,BERT-based and DistilBERT-based NER models; ML...,Entity extraction in hereditary disease-relate...,BERT-based NER achieved high F1-scores for gen...,None,None
3,A Comparative Sentiment Analysis of Greek Clin...,In addressing the critical role of emotional c...,PubMed,1.0,"BERT, RoBERTa, GPT-2, XLNet; ML Task: Sentimen...",Sentiment analysis of patient-clinician conver...,Importance of different language incorporation...,None,None
6,A Systematic Review of Genetics- and Molecular...,The process of identification and management o...,PubMed,1.0,"ML models (but simple, not transformers) using...",Diagnosis of neurological disorders using gene...,Reviews the use of genetic-based ML models for...,None,None
10,A comparative evaluation of ChatGPT 3.5 and Ch...,To evaluate the efficacy of ChatGPT 4 (GPT-4) ...,PubMed,2.0,"GPT-3.5, GPT-4; ML Task: Genetic information d...","Genetic conditions (BRCA1, HFE, MLH1) and gene...",GPT-4 performs better than GPT-3.5 in genetic ...,1,"edu, disc"
12,A foundational large language model for edible...,Significant progress has been made in the fiel...,"PubMed,biorxiv",1.0,Large language model (AgroNT); ML Task: Genomi...,Genomic prediction for plant species using DNA...,AgroNT predicts regulatory annotations and fun...,None,None
17,Accurate proteome-wide missense variant effect...,None,None,1,None,None,None,1,ana: AVE
25,AntiFormer: graph enhanced large language mode...,Antibodies play a pivotal role in immune defen...,PubMed,2.0,Graph-based LLM (AntiFormer); ML Task: Predict...,Antibody binding affinity prediction for thera...,AntiFormer accurately predicts antibody bindin...,1,ana: AVE
26,Approximating facial expression effects on dia...,Artificial intelligence (AI) is increasingly u...,PubMed,1.0,"Generative AI (GAN-based, HyperStyle + StyleGA...","Diagnosis of genetic syndromes (Williams, Ang...",generative AI's role in altering facial expres...,None,None
30,BERT Based Clinical Knowledge Extraction for B...,"Background : Knowledge is evolving over time, ...",arXiv,2.0,"BERT, CRF layer; ML Task: Named entity recogni...",Biomedical knowledge extraction for constructi...,BERT-based NER and relation extraction achieve...,1,pre: KNLR
